# 🐋 Kuramoto Model Simulation for Whale Vocal Synchrony (Group G1)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [3]:
df = pd.read_csv("PrepData.csv")
df["CallType"] = df["CallType"].fillna("InitiationCall")
df_g1 = df[df["Group"] == "G1"].copy()
df_g1 = df_g1.dropna(subset=["WhaleID", "StartTime"])

In [4]:
whales = df_g1["WhaleID"].unique()
n_whales = len(whales)
whale_to_idx = {whale: i for i, whale in enumerate(whales)}

In [5]:
call_intervals = []
for whale, group in df_g1.groupby("WhaleID"):
    times = group.sort_values("StartTime")["StartTime"].values
    if len(times) > 1:
        intervals = np.diff(times)
        call_intervals.append((whale, 1 / np.mean(intervals)))
    else:
        call_intervals.append((whale, 1.0))
omega = np.array([dict(call_intervals).get(whale, 1.0) for whale in whales])

In [6]:
A = np.zeros((n_whales, n_whales))
for bout_id, bout_df in df_g1.groupby("bout"):
    bout_sorted = bout_df.sort_values("StartTime")
    initiator_row = bout_sorted[bout_sorted["CallType"] == "InitiationCall"]
    responders = bout_sorted[bout_sorted["CallType"] == "ResponseCall"]
    if not initiator_row.empty:
        initiator = initiator_row.iloc[0]["WhaleID"]
        for _, responder_row in responders.iterrows():
            responder = responder_row["WhaleID"]
            if initiator != responder:
                i = whale_to_idx[responder]
                j = whale_to_idx[initiator]
                A[i, j] += 1
A /= A.max()

In [7]:
T = 100
dt = 0.1
steps = int(T / dt)
K = 2.0
theta = np.zeros((steps, n_whales))
theta[0] = np.random.uniform(0, 2 * np.pi, n_whales)
for t in range(1, steps):
    dtheta = omega + (K / n_whales) * np.sum(A * np.sin(theta[t-1] - theta[t-1][:, None]), axis=1)
    theta[t] = theta[t-1] + dt * dtheta

In [8]:
r = np.abs(np.mean(np.exp(1j * theta), axis=1))
time = np.linspace(0, T, steps)

In [9]:
kuramoto_data = {
    "whales": whales.tolist(),
    "omega": omega.tolist(),
    "A": A.tolist(),
    "theta": theta.tolist(),
    "r": r.tolist(),
    "time": time.tolist()
}

with open("kuramoto_simulation_G1.json", "w") as f:
    json.dump(kuramoto_data, f)